# Tensorflow2.0 小练习

In [119]:
import numpy as np
import tensorflow as tf

@tf.function
def func(x):
    x_exp = tf.math.exp(x)
    print(x_exp)
    out = x_exp+1
    return out

In [121]:
func(tf.constant(0.1, dtype=tf.float32))
func(tf.constant(1., dtype=tf.float32))
print()

In [113]:
import tensorflow as tf
import numpy as np

# 能用到的API

In [1]:
import tensorflow as tf
tf.math.exp()
tf.math.log()
tf.reduce_sum()
tf.reduce_mean()

## 实现softmax函数

In [6]:
import numpy as np
print(tf.math.exp([1, 2, 3, 4.]))
print(np.exp([1, 2, 3, 4.]))

@tf.function
def func():
    print(tf.math.exp([1, 2, 3, 4.]))
func()

test_data = np.random.normal(size=[10, 5])
print(test_data)
print('--'*20)
print(tf.reduce_sum(test_data, axis=1, keepdims=True)) # [N, 1]

tf.Tensor([ 2.7182817  7.389056  20.085537  54.59815  ], shape=(4,), dtype=float32)
[ 2.71828183  7.3890561  20.08553692 54.59815003]
Tensor("Exp:0", shape=(4,), dtype=float32)
[[ 0.56577835  0.03925728  0.12253202  0.42823133 -0.30519943]
 [-0.62723232 -0.77811349 -0.51248257 -0.08009214  0.45644417]
 [-0.47520551  1.20698361  0.35421463  1.51033826  0.31313441]
 [ 0.77509969  0.17709761  0.51255336  0.70945003  0.58216116]
 [ 0.18728327 -0.60192896  1.2517836   0.09139799 -0.43010604]
 [-0.49701167  1.5995401  -0.69059412 -0.06869925 -3.53611644]
 [-1.13273144  0.85586692  0.11808564 -1.48678667  1.77579503]
 [ 0.18899566  1.09708078 -0.07208181 -1.89496327  1.29759657]
 [-1.25227742 -0.54591516 -0.44185156 -0.51002137 -0.5215976 ]
 [ 0.7112895  -1.19476881  0.26648144  0.62118595 -1.96629359]]
----------------------------------------
tf.Tensor(
[[ 0.85059956]
 [-1.54147636]
 [ 2.90946539]
 [ 2.75636184]
 [ 0.49842986]
 [-3.19288139]
 [ 0.13022949]
 [ 0.61662792]
 [-3.27166311]
 [-1.

# softmax function: $\frac{e^{x_i}}{\sum_{j=1}^{d}{e^{x_j}}}$
# $\sum_{i}{\frac{e^{x_i}}{\sum_{j=1}^{d}{e^{x_j}}}} == 1$

In [10]:
def softmax(x):
    epsilon = 1e-12
    '''shape of x [N, d]'''
    x_exp = tf.math.exp(x) # shape=[N, d]
    denominator = tf.reduce_sum(x_exp, axis=1, keepdims=True) #shape=[N, 1]
    prob_x = x_exp/(denominator+epsilon)
    return prob_x

test_data = np.random.normal(size=[10, 5])
(softmax(test_data).numpy() - tf.nn.softmax(test_data, axis=-1).numpy())**2 <0.000001

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

## 实现sigmoid函数

# $\frac{1}{1+e^{-x}}$

In [12]:
def sigmoid(x):
    '''shape=[N, d]'''
    x_exp = tf.math.exp(-x)
#     prob_x = 1/(1+x_exp)
    prob_x = tf.divide(1, tf.add(1, x_exp))
    return prob_x

test_data = np.random.normal(size=[10, 5])
(sigmoid(test_data).numpy() - tf.nn.sigmoid(test_data).numpy())**2 < 0.0001

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

## 实现 softmax 交叉熵loss函数
# $\sum_{c}{-p(y=c|x)*log(p(\hat{y}=c|x))}$
# $s_{ij} = a_{ij}*b_{ij}$ 
# $s_{ik} = \sum_j{a_{ij}*b_{jk}}$ matmul

In [125]:
def softmax_ce(p, label):
    '''
        x shape = [N, c] ===> probability
        label shape = [N, c]
    '''
    loss = tf.reduce_mean(tf.reduce_sum(-label*tf.math.log(p), axis=1))
    return loss

test_data = np.random.normal(size=[10, 5])
prob = tf.nn.softmax(test_data)
label = np.zeros_like(test_data)
label[np.arange(10), np.random.randint(0, 5, size=10)]=1.

((tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(label, test_data))
  - softmax_ce(prob, label))**2 < 0.0001).numpy()


True

## 实现 sigmoid 交叉熵loss函数

# $-p(y=1|x)*log(p(\hat{y}=1|x)-p(y=0|x)*log(p(\hat{y}=0|x))$

In [122]:
def sigmoid_ce(p, label):
    epsilon = 1e-12
    '''shape = [N,]'''
    losses = -label * tf.math.log(p+epsilon) - (1.-label) * tf.math.log(1.-p+epsilon) # shape=[N,]
    loss = tf.reduce_mean(losses)
    return loss

test_data = np.random.normal(size=[10])
prob = tf.nn.sigmoid(test_data)
label = np.random.randint(0, 2, 10).astype(test_data.dtype)
print (label)

((tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(label, test_data))- sigmoid_ce(prob, label))**2 < 0.0001).numpy()


W0824 13:53:24.909731 4684809664 deprecation.py:323] From /Users/jerrik/a3/envs/tf2.0/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:182: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]


True

In [51]:
# @tf.function
def func(x):
    x_exp = tf.math.exp(x)
    out = x_exp+1
    return out

func(0.1)

tf.Tensor(1.105171, shape=(), dtype=float32)


<tf.Tensor: id=1395, shape=(), dtype=float32, numpy=2.105171>

array([ 2.7182817,  7.389056 , 20.085537 , 54.59815  ], dtype=float32)